In [22]:
from pyspark.sql import SparkSession, dataframe
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType
from pyspark.sql import HiveContext
from pyspark.sql.functions import *
from pyspark.sql import functions as f
import re

spark = SparkSession.builder.master("local[*]")\
    .enableHiveSupport()\
    .getOrCreate()

In [23]:
# Passando configurações gerais para os .csv
config_csv = {
  "delimiter" : ";"
  ,"encode" : "utf-8"
  ,"header" : True
  ,"inferSchema" : True
}
# Passando os path em comum para os .csv
root_path = "/user/amadeus/dados_processamento/dados/"

In [24]:
# Criando o dataframe Clientes

df_clientes = spark.read.options(**config_csv).csv(f'{root_path}CLIENTES.csv')

# Vendo os tipos das colunas da tabela cliente
# df_clientes.printSchema()

# Tratando linha Line of Business que estava com três espaços '   ' para 'Nao informado' por ser string.
df_clientes = df_clientes.withColumn('Line of Business', regexp_replace('Line of Business', '   ', 'Nao informado'))

df_clientes.toPandas()

,Address Number,Business Family,Business Unit,Customer,CustomerKey,Customer Type,Division,Line of Business,Phone,Region Code,Regional Sales Mgr,Search Type
0,10000000,R3,1,City Supermarket,10000000,G2,2,Nao informado,816-455-8733,4,S16,C
1,10000453,R3,1,A Supermarket,10000453,G1,1,Nao informado,816-455-8733,5,S19,C
2,10000455,R3,1,Caribian Supermarket,10000455,G2,2,Nao informado,816-455-8733,1,S16,C
3,10000456,R1,1,A&B Shop,10000456,G3,1,Nao informado,816-455-8733,0,S2,C
4,10000457,O2,1,A&G Shop,10000457,G1,1,Nao informado,816-455-8733,5,S1,C
...,...,...,...,...,...,...,...,...,...,...,...,...
679,10027560,R2,1,Zilog Shop,10027560,G2,2,Nao informado,816-455-8733,2,S1,C
680,10027572,R3,1,ZipLip.Com Shop,10027572,G2,2,Nao informado,816-455-8733,3,S1,C
681,10027575,R3,1,Zitel Shop,10027575,G2,2,Nao informado,816-455-8733,2,S1,C
682,10027583,R2,1,zNET Shop,10027583,G2,2,Nao informado,816-455-8733,4,S5,C


In [25]:
# Criando o dataframe Divisao

df_divisao = spark.read.options(**config_csv).csv(f'{root_path}DIVISAO.csv')

df_divisao.printSchema()

root
 |-- Division: integer (nullable = true)
 |-- Division Name: string (nullable = true)



In [26]:
# Criando o dataframe Endereço

df_endereco = spark.read.options(**config_csv).csv(f'{root_path}ENDERECO.csv')

# Tratando as colunas sujas com espaço em branco e passando a informação de 'Nao informado para as colunas vazias após tratamento'

for column in df_endereco.columns:
    df_endereco = df_endereco.withColumn(column, trim(df_endereco[column]))
    df_endereco = df_endereco.withColumn(column, when(df_endereco[column] == '', "Nao informado")\
                                         .when(df_endereco[column].isNull(), "Nao informado")\
                                         .otherwise(df_endereco[column]))
    
df_endereco = df_endereco.withColumn("Address Number", col("Address Number").cast('integer'))
    
df_endereco.toPandas()

,Address Number,City,Country,Customer Address 1,Customer Address 2,Customer Address 3,Customer Address 4,State,Zip Code
0,10000000,Akron,US,PO Box 6258,Nao informado,Nao informado,Nao informado,OH,44312
1,10000453,Nao informado,UK,Nao informado,Nao informado,Nao informado,Nao informado,Nao informado,Nao informado
2,10000455,Huntington Beach,US,7392 Count Circle,Nao informado,Nao informado,Nao informado,CA,92647
3,10000456,Edmonton,CA,8151 Wagner Road,Nao informado,Nao informado,Nao informado,AB,T6E 4N6
4,10000458,Saginaw,US,PO Box 840,Nao informado,Nao informado,Nao informado,MI,48606
...,...,...,...,...,...,...,...,...,...
450,10027560,Odessa,US,3356 Kermit Highway,Nao informado,Nao informado,Nao informado,TX,79764
451,10027572,Elma,US,2210 Bowen Road,Nao informado,Nao informado,Nao informado,NY,14059
452,10027575,Dallas,US,10400 Plano Road,Nao informado,Nao informado,Nao informado,TX,75238
453,10027583,Morton,US,Attention: Charlene Hoyer,500 North Morton Avenue,PO Box 474,Nao informado,IL,61550-0474


In [27]:
df_endereco.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in df_endereco.columns]).toPandas()

,Address Number,City,Country,Customer Address 1,Customer Address 2,Customer Address 3,Customer Address 4,State,Zip Code
0,0,0,0,0,0,0,0,0,0


In [28]:
df_endereco.printSchema()

root
 |-- Address Number: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Customer Address 1: string (nullable = true)
 |-- Customer Address 2: string (nullable = true)
 |-- Customer Address 3: string (nullable = true)
 |-- Customer Address 4: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zip Code: string (nullable = true)



In [29]:
# Criando o dataframe Região

df_regiao = spark.read.options(**config_csv).csv(f'{root_path}REGIAO.csv')

In [30]:
# Criando o dataframe Vendas 

df_vendas = spark.read.options(**config_csv).csv(f'{root_path}VENDAS.csv')

df_vendas.toPandas() # Pelo df vemos que será necessário excluir as linhas vazias com None e Nan da tabela.

,Actual Delivery Date,CustomerKey,DateKey,Discount Amount,Invoice Date,Invoice Number,Item Class,Item Number,Item,Line Number,...,Order Number,Promised Delivery Date,Sales Amount,Sales Amount Based on List Price,Sales Cost Amount,Sales Margin Amount,Sales Price,Sales Quantity,Sales Rep,U/M
0,28/04/2019,10000481.0,28/04/2018,"-237,91",30/04/2018,100012.0,None,None,Urban Large Eggs,2000.0,...,200015.0,28/04/2019,"237,91",0,0,"237,91","237,91",1.0,184.0,EA
1,12/07/2019,10002220.0,12/07/2018,"368,79",14/07/2018,100233.0,P01,20910,Moms Sliced Turkey,1000.0,...,200245.0,12/07/2019,"456,17","824,96",0,"456,17","456,17",1.0,127.0,EA
2,14/10/2019,10002220.0,15/10/2018,"109,73",17/10/2018,116165.0,P01,38076,Cutting Edge Foot-Long Hot Dogs,1000.0,...,213157.0,14/10/2019,"438,93","548,66",0,"438,93","438,93",1.0,127.0,EA
3,01/06/2019,10002489.0,01/06/2018,"-211,75",03/06/2018,100096.0,None,None,Kiwi Lox,1000.0,...,200107.0,01/06/2019,"211,75",0,0,"211,75","211,75",1.0,160.0,EA
4,26/05/2019,10004516.0,25/05/2018,"96627,94",27/05/2018,103341.0,P01,60776,High Top Sweet Onion,1000.0,...,203785.0,26/05/2019,"89248,66","185876,6",0,"89248,66","196,1509011",455.0,124.0,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65530,None,NaN,None,None,None,NaN,None,None,None,NaN,...,NaN,None,None,None,None,None,None,NaN,NaN,None
65531,None,NaN,None,None,None,NaN,None,None,None,NaN,...,NaN,None,None,None,None,None,None,NaN,NaN,None
65532,None,NaN,None,None,None,NaN,None,None,None,NaN,...,NaN,None,None,None,None,None,None,NaN,NaN,None
65533,None,NaN,None,None,None,NaN,None,None,None,NaN,...,NaN,None,None,None,None,None,None,NaN,NaN,None


In [31]:
# Exibindo tipos dos campos de cliente pois teremos de alterar o tipo de DateKey para date (antes como String)

df_vendas.printSchema()

root
 |-- Actual Delivery Date: string (nullable = true)
 |-- CustomerKey: integer (nullable = true)
 |-- DateKey: string (nullable = true)
 |-- Discount Amount: string (nullable = true)
 |-- Invoice Date: string (nullable = true)
 |-- Invoice Number: integer (nullable = true)
 |-- Item Class: string (nullable = true)
 |-- Item Number: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Line Number: integer (nullable = true)
 |-- List Price: string (nullable = true)
 |-- Order Number: integer (nullable = true)
 |-- Promised Delivery Date: string (nullable = true)
 |-- Sales Amount: string (nullable = true)
 |-- Sales Amount Based on List Price: string (nullable = true)
 |-- Sales Cost Amount: string (nullable = true)
 |-- Sales Margin Amount: string (nullable = true)
 |-- Sales Price: string (nullable = true)
 |-- Sales Quantity: integer (nullable = true)
 |-- Sales Rep: integer (nullable = true)
 |-- U/M: string (nullable = true)



In [32]:
# Alterando os tipos das colunas para os seus respectivos e corretos tipos

# Usando o dtypes que retorna o o nome da coluna e o tipo

for i in df_vendas.dtypes:
    if re.search("Date", i[0]):
        df_vendas = df_vendas\
                    .withColumn(
                        i[0], 
                        to_date(col(i[0]), 'dd/MM/yyyy')
                        .alias(i[0])
                        )

vendastodouble = ['Discount Amount', 'List Price', 'Sales Amount', 
                'Sales Amount Based on List Price', 'Sales Cost Amount',
                 'Sales Margin Amount', 'Sales Price']

for c in vendastodouble:
    df_vendas = df_vendas.withColumn(c, regexp_replace(c, '\,', '.'))
    df_vendas = df_vendas.withColumn(c, col(c).cast('double'))
                            
    
df_vendas.printSchema()

root
 |-- Actual Delivery Date: date (nullable = true)
 |-- CustomerKey: integer (nullable = true)
 |-- DateKey: date (nullable = true)
 |-- Discount Amount: double (nullable = true)
 |-- Invoice Date: date (nullable = true)
 |-- Invoice Number: integer (nullable = true)
 |-- Item Class: string (nullable = true)
 |-- Item Number: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Line Number: integer (nullable = true)
 |-- List Price: double (nullable = true)
 |-- Order Number: integer (nullable = true)
 |-- Promised Delivery Date: date (nullable = true)
 |-- Sales Amount: double (nullable = true)
 |-- Sales Amount Based on List Price: double (nullable = true)
 |-- Sales Cost Amount: double (nullable = true)
 |-- Sales Margin Amount: double (nullable = true)
 |-- Sales Price: double (nullable = true)
 |-- Sales Quantity: integer (nullable = true)
 |-- Sales Rep: integer (nullable = true)
 |-- U/M: string (nullable = true)



In [33]:
# DF vendas com valores alterados
df_vendas.toPandas()

,Actual Delivery Date,CustomerKey,DateKey,Discount Amount,Invoice Date,Invoice Number,Item Class,Item Number,Item,Line Number,...,Order Number,Promised Delivery Date,Sales Amount,Sales Amount Based on List Price,Sales Cost Amount,Sales Margin Amount,Sales Price,Sales Quantity,Sales Rep,U/M
0,2019-04-28,10000481.0,2018-04-28,-237.91,2018-04-30,100012.0,None,None,Urban Large Eggs,2000.0,...,200015.0,2019-04-28,237.91,0.00,0.0,237.91,237.910000,1.0,184.0,EA
1,2019-07-12,10002220.0,2018-07-12,368.79,2018-07-14,100233.0,P01,20910,Moms Sliced Turkey,1000.0,...,200245.0,2019-07-12,456.17,824.96,0.0,456.17,456.170000,1.0,127.0,EA
2,2019-10-14,10002220.0,2018-10-15,109.73,2018-10-17,116165.0,P01,38076,Cutting Edge Foot-Long Hot Dogs,1000.0,...,213157.0,2019-10-14,438.93,548.66,0.0,438.93,438.930000,1.0,127.0,EA
3,2019-06-01,10002489.0,2018-06-01,-211.75,2018-06-03,100096.0,None,None,Kiwi Lox,1000.0,...,200107.0,2019-06-01,211.75,0.00,0.0,211.75,211.750000,1.0,160.0,EA
4,2019-05-26,10004516.0,2018-05-25,96627.94,2018-05-27,103341.0,P01,60776,High Top Sweet Onion,1000.0,...,203785.0,2019-05-26,89248.66,185876.60,0.0,89248.66,196.150901,455.0,124.0,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65530,None,NaN,None,NaN,None,NaN,None,None,None,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
65531,None,NaN,None,NaN,None,NaN,None,None,None,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
65532,None,NaN,None,NaN,None,NaN,None,None,None,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
65533,None,NaN,None,NaN,None,NaN,None,None,None,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None


In [34]:
# Retirando as variáveis "PK" DateKey (dia da venda) nulas da tabela

df_vendas = df_vendas.filter(col("CustomerKey").isNotNull())
df_vendas.toPandas()

,Actual Delivery Date,CustomerKey,DateKey,Discount Amount,Invoice Date,Invoice Number,Item Class,Item Number,Item,Line Number,...,Order Number,Promised Delivery Date,Sales Amount,Sales Amount Based on List Price,Sales Cost Amount,Sales Margin Amount,Sales Price,Sales Quantity,Sales Rep,U/M
0,2019-04-28,10000481,2018-04-28,-237.91,2018-04-30,100012,None,None,Urban Large Eggs,2000,...,200015,2019-04-28,237.91,0.00,0.00,237.91,237.910000,1,184,EA
1,2019-07-12,10002220,2018-07-12,368.79,2018-07-14,100233,P01,20910,Moms Sliced Turkey,1000,...,200245,2019-07-12,456.17,824.96,0.00,456.17,456.170000,1,127,EA
2,2019-10-14,10002220,2018-10-15,109.73,2018-10-17,116165,P01,38076,Cutting Edge Foot-Long Hot Dogs,1000,...,213157,2019-10-14,438.93,548.66,0.00,438.93,438.930000,1,127,EA
3,2019-06-01,10002489,2018-06-01,-211.75,2018-06-03,100096,None,None,Kiwi Lox,1000,...,200107,2019-06-01,211.75,0.00,0.00,211.75,211.750000,1,160,EA
4,2019-05-26,10004516,2018-05-25,96627.94,2018-05-27,103341,P01,60776,High Top Sweet Onion,1000,...,203785,2019-05-26,89248.66,185876.60,0.00,89248.66,196.150901,455,124,SE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65277,2020-03-18,10017638,2019-03-18,505.78,2019-03-21,226497,P01,13447,High Top Oranges,8000,...,320895,2020-03-18,569.90,1075.68,239.95,329.95,63.322222,9,180,EA
65278,2020-03-18,10017638,2019-03-18,410.75,2019-03-21,226497,P01,25906,Landslide White Sugar,38000,...,320895,2020-03-18,462.81,873.56,423.55,39.26,231.405000,2,180,EA
65279,2020-03-18,10017638,2019-03-18,876.16,2019-03-21,226497,P01,61856,Moms Potato Salad,227001,...,320895,2020-03-18,987.20,1863.36,574.00,413.20,123.400000,8,180,EA
65280,2020-03-18,10017638,2019-03-18,24226.77,2019-03-21,226498,P01,17801,Better Fancy Canned Sardines,1000,...,320907,2020-03-18,27297.51,51524.28,16188.90,11108.61,758.264167,36,180,EA


In [35]:
# Usando o dtypes mais uma vez para preencher os campos nulos, o i[1] representa o tipo
# Caso seja int, preencher com 0, caso seja string preencher com Nao informado

for i in df_vendas.dtypes:
    if re.search("int", i[1]):
      df_vendas = df_vendas.na.fill(0)
    if re.search("string", i[1]):
      df_vendas = df_vendas.na.fill("Nao informado")
df_vendas.limit(5).toPandas()

,Actual Delivery Date,CustomerKey,DateKey,Discount Amount,Invoice Date,Invoice Number,Item Class,Item Number,Item,Line Number,...,Order Number,Promised Delivery Date,Sales Amount,Sales Amount Based on List Price,Sales Cost Amount,Sales Margin Amount,Sales Price,Sales Quantity,Sales Rep,U/M
0,2019-04-28,10000481,2018-04-28,-237.91,2018-04-30,100012,Nao informado,Nao informado,Urban Large Eggs,2000,...,200015,2019-04-28,237.91,0.00,0.0,237.91,237.910000,1,184,EA
1,2019-07-12,10002220,2018-07-12,368.79,2018-07-14,100233,P01,20910,Moms Sliced Turkey,1000,...,200245,2019-07-12,456.17,824.96,0.0,456.17,456.170000,1,127,EA
2,2019-10-14,10002220,2018-10-15,109.73,2018-10-17,116165,P01,38076,Cutting Edge Foot-Long Hot Dogs,1000,...,213157,2019-10-14,438.93,548.66,0.0,438.93,438.930000,1,127,EA
3,2019-06-01,10002489,2018-06-01,-211.75,2018-06-03,100096,Nao informado,Nao informado,Kiwi Lox,1000,...,200107,2019-06-01,211.75,0.00,0.0,211.75,211.750000,1,160,EA
4,2019-05-26,10004516,2018-05-25,96627.94,2018-05-27,103341,P01,60776,High Top Sweet Onion,1000,...,203785,2019-05-26,89248.66,185876.60,0.0,89248.66,196.150901,455,124,SE


In [36]:
# Observando se alguma linha está nula

df_vendas.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in df_vendas.columns]).toPandas()

,Actual Delivery Date,CustomerKey,DateKey,Discount Amount,Invoice Date,Invoice Number,Item Class,Item Number,Item,Line Number,...,Order Number,Promised Delivery Date,Sales Amount,Sales Amount Based on List Price,Sales Cost Amount,Sales Margin Amount,Sales Price,Sales Quantity,Sales Rep,U/M
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# Respondendo o exercício:

# Quantos pedidos foram realizados?

# Quantos clientes tem em nossa base?

# Quantos clientes temos por Região?

# Quantidade de vendas em 2018.

In [38]:
# Question #1: Quantos pedidos foram realizados?
# Answer: Observar a tabela vendas e fazer o count * da tabelas vendas e contando os itens totais de venda
# Answer bonus: Verificar quantidade de itens vendidos nas vendas, somando a coluna SalesQuantity
df_vendas.createOrReplaceTempView("vendasView")
df_vendas_pedidos = spark\
                    .sql("""
                        SELECT COUNT(*) as Total_Vendas, sum(`Sales Quantity`) as Quantitade_Itens_Vendidos
                        FROM VendasView
                    """)
df_vendas_pedidos.toPandas()

,Total_Vendas,Quantitade_Itens_Vendidos
0,65282,2943194


In [39]:
df_clientes.createOrReplaceTempView("clientesView")
spark.sql("""
        SELECT * FROM clientesView
""").toPandas()

,Address Number,Business Family,Business Unit,Customer,CustomerKey,Customer Type,Division,Line of Business,Phone,Region Code,Regional Sales Mgr,Search Type
0,10000000,R3,1,City Supermarket,10000000,G2,2,Nao informado,816-455-8733,4,S16,C
1,10000453,R3,1,A Supermarket,10000453,G1,1,Nao informado,816-455-8733,5,S19,C
2,10000455,R3,1,Caribian Supermarket,10000455,G2,2,Nao informado,816-455-8733,1,S16,C
3,10000456,R1,1,A&B Shop,10000456,G3,1,Nao informado,816-455-8733,0,S2,C
4,10000457,O2,1,A&G Shop,10000457,G1,1,Nao informado,816-455-8733,5,S1,C
...,...,...,...,...,...,...,...,...,...,...,...,...
679,10027560,R2,1,Zilog Shop,10027560,G2,2,Nao informado,816-455-8733,2,S1,C
680,10027572,R3,1,ZipLip.Com Shop,10027572,G2,2,Nao informado,816-455-8733,3,S1,C
681,10027575,R3,1,Zitel Shop,10027575,G2,2,Nao informado,816-455-8733,2,S1,C
682,10027583,R2,1,zNET Shop,10027583,G2,2,Nao informado,816-455-8733,4,S5,C


In [40]:
# Question #2: Quantos clientes tem em nossa base?
# Answer: Contar o * de PK de clientes

spark.sql("""
        SELECT COUNT (*) as Tot_linhas, COUNT(DISTINCT CustomerKey) as Tot_PKCustomer FROM clientesView
""").toPandas()

,Tot_linhas,Tot_PKCustomer
0,684,683


In [41]:
# Verificando o cliente duplicado

spark.sql("""
        SELECT CustomerKey, Count(CustomerKey) AS TotCustomer
        FROM clientesView 
        GROUP BY CustomerKey
        ORDER BY TotCustomer DESC
""").limit(3).toPandas()

,CustomerKey,TotCustomer
0,10021911,2
1,10025039,1
2,10022249,1


In [42]:
# A diferença está no Customer Type, mas temos de considerar apenas um deles

spark.sql("""
        SELECT *
        FROM clientesView 
        WHERE CustomerKey = '10021911'
""").toPandas()

,Address Number,Business Family,Business Unit,Customer,CustomerKey,Customer Type,Division,Line of Business,Phone,Region Code,Regional Sales Mgr,Search Type
0,10021911,R3,1,PING Shop,10021911,G1,2,M1,816-455-8733,2,S19,C
1,10021911,R3,1,Ping Shop,10021911,G2,2,M1,816-455-8733,2,S16,C


In [43]:
# Calculando o total de clientes após o distinct

df_clientes = df_clientes.dropDuplicates(["CustomerKey"])

print("Total de clientes após o distinct: " + str(df_clientes.count()))

df_clientes.toPandas()

Total de clientes após o distinct: 683


,Address Number,Business Family,Business Unit,Customer,CustomerKey,Customer Type,Division,Line of Business,Phone,Region Code,Regional Sales Mgr,Search Type
0,10013312,R2,1,Gate9th Shop,10013312,G2,1,Nao informado,816-455-8733,5,S9,C
1,10018352,R3,1,Johnson Store,10018352,G3,1,Nao informado,816-455-8733,5,S5,C
2,10022746,R3,1,Rdlabs Shop,10022746,G1,1,M1,816-455-8733,5,S19,C
3,10024916,R3,1,T Market,10024916,G2,1,Nao informado,816-455-8733,5,S9,C
4,10025249,R1,1,Texasgulf Market,10025249,G2,2,Nao informado,816-455-8733,2,S11,C
...,...,...,...,...,...,...,...,...,...,...,...,...
678,10025051,R3,1,Taos Supermarket,10025051,G2,2,Nao informado,816-455-8733,1,S16,C
679,10011521,R2,1,Farr Market,10011521,G2,2,Nao informado,816-455-8733,2,S5,C
680,10013376,R3,1,GCC Store,10013376,G3,1,Nao informado,816-455-8733,5,S5,C
681,10024248,R3,1,Sifton Supermarket,10024248,G2,2,M1,816-455-8733,1,S16,C


In [44]:
# Question #3: Quantos clientes temos por Região?
# Answer: Vejo as chaves que interligam a tabela (Region Code)
df_clientes.printSchema()
df_regiao.printSchema()

root
 |-- Address Number: integer (nullable = true)
 |-- Business Family: string (nullable = true)
 |-- Business Unit: integer (nullable = true)
 |-- Customer: string (nullable = true)
 |-- CustomerKey: integer (nullable = true)
 |-- Customer Type: string (nullable = true)
 |-- Division: integer (nullable = true)
 |-- Line of Business: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Region Code: integer (nullable = true)
 |-- Regional Sales Mgr: string (nullable = true)
 |-- Search Type: string (nullable = true)

root
 |-- Region Code: integer (nullable = true)
 |-- Region Name: string (nullable = true)



In [45]:
# Crio as views para trabalhar em SQL
df_clientes.createOrReplaceTempView("clientesView")
df_regiao.createOrReplaceTempView("regiaoView")
clientes_regiao = spark\
                    .sql("""
                        SELECT r.`Region Name`, count(Customer) AS Tot_Clientes
                        FROM clientesView c
                        INNER JOIN regiaoView r
                        ON c.`Region Code` = r.`Region Code`
                        GROUP BY r.`Region Name`
                        ORDER BY tot_clientes DESC
                    """)
clientes_regiao.toPandas()
# clientes_regiao.show()
# display(clientes_regiao) no Databricks

,Region Name,Tot_Clientes
0,International,299
1,Central,117
2,Southern,99
3,Western,89
4,Northeast,42
5,Canada,37


In [46]:
# Question #4: Qual a quantidade de vendas em 2018?
# Answer: Puxo uma seleção de count com o DateKey == 2018 (Ano de venda)
df_vendas2018 = spark\
                    .sql("""
                        SELECT COUNT(*) Vendas_2018
                        FROM VendasView
                        WHERE YEAR(DateKey) = 2018
                    """)

df_vendas2018.toPandas()

,Vendas_2018
0,30560
